<a href="https://colab.research.google.com/github/microhum/thaifaces-scraping/blob/main/webscraping/TTF_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/free-thai-font-collection.zip /content/free-thai-font-collection.zip
!cp /content/drive/MyDrive/ThaiVecFont/scrape_ttf.zip /content/scrape_ttf.zip

In [ ]:
!unzip -o /content/free-thai-font-collection.zip -d /content/free-thai-font-collection/
!unzip -o /content/scrape_ttf.zip -d /content/scrape_ttf/

Archive:  /content/free-thai-font-collection.zip
 extracting: /content/free-thai-font-collection/cadson-demak/.uuid  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-Bold.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-ExtraLight.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-Light.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-Medium.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-Regular.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Athiti-SemiBold.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Chonburi-Regular.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Itim-Regular.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Kanit-Black.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Kanit-BlackItalic.ttf  
  inflating: /content/free-thai-font-collection/cadson-demak/Kan

In [ ]:
import os
import shutil

font_paths = []
for root, dirs, files in os.walk("/content/free-thai-font-collection"):
    for file_ in files:
      if file_[-3:] in ["ttf"]:
        file_path = root+"/"+file_
        font_paths.append(file_path)

for root, dirs, files in os.walk("/content/scrape_ttf"):
    for file_ in files:
      if file_[-3:] in ["ttf"]:
        file_path = root+"/"+file_
        font_paths.append(file_path)

In [ ]:
len(font_paths)

2393

In [ ]:
!rm -rf "/content/Thai_ttf/"
!mkdir "/content/Thai_ttf/"
for idx, item in enumerate(font_paths):
  shutil.copy(item, f"/content/Thai_ttf/")

In [ ]:
len(os.listdir("/content/Thai_ttf/"))

2334

## Remove duplicate files

In [ ]:
#!/usr/bin/env python3
from collections import defaultdict
from tqdm import tqdm
import hashlib
import os
import sys


def chunk_reader(fobj, chunk_size=1024):
    """Generator that reads a file in chunks of bytes"""
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk


def get_hash(filename, first_chunk_only=False, hash=hashlib.sha1):
    hashobj = hash()
    file_object = open(filename, 'rb')

    if first_chunk_only:
        hashobj.update(file_object.read(1024))
    else:
        for chunk in chunk_reader(file_object):
            hashobj.update(chunk)
    hashed = hashobj.digest()

    file_object.close()
    return hashed


def check_for_duplicates(paths, hash=hashlib.sha1):
    hashes_by_size = defaultdict(list)  # dict of size_in_bytes: [full_path_to_file1, full_path_to_file2, ]
    hashes_on_1k = defaultdict(list)  # dict of (hash1k, size_in_bytes): [full_path_to_file1, full_path_to_file2, ]
    hashes_full = {}   # dict of full_file_hash: full_path_to_file_string
    idx = 0

    for path in paths:
        for dirpath, dirnames, filenames in tqdm(os.walk(path)):
            # get all files that have the same size - they are the collision candidates
            for filename in tqdm(filenames):
                full_path = os.path.join(dirpath, filename)
                try:
                    # if the target is a symlink (soft one), this will
                    # dereference it - change the value to the actual target file
                    full_path = os.path.realpath(full_path)
                    file_size = os.path.getsize(full_path)
                    hashes_by_size[file_size].append(full_path)
                except (OSError,):
                    # not accessible (permissions, etc) - pass on
                    continue


    # For all files with the same file size, get their hash on the 1st 1024 bytes only
    for size_in_bytes, files in hashes_by_size.items():
        if len(files) < 2:
            continue    # this file size is unique, no need to spend CPU cycles on it

        for filename in files:
            try:
                small_hash = get_hash(filename, first_chunk_only=True)
                # the key is the hash on the first 1024 bytes plus the size - to
                # avoid collisions on equal hashes in the first part of the file
                # credits to @Futal for the optimization
                hashes_on_1k[(small_hash, size_in_bytes)].append(filename)
            except (OSError,):
                # the file access might've changed till the exec point got here
                continue

    # For all files with the hash on the 1st 1024 bytes, get their hash on the full file - collisions will be duplicates
    for __, files_list in hashes_on_1k.items():
        if len(files_list) < 2:
            continue    # this hash of fist 1k file bytes is unique, no need to spend cpy cycles on it

        for filename in files_list:
            try:
                full_hash = get_hash(filename, first_chunk_only=False)
                duplicate = hashes_full.get(full_hash)
                if duplicate:
                    idx += 1
                    print("idx {} Duplicate found: {} and {}".format(str(idx), filename, duplicate))
                    os.remove(duplicate)
                else:
                    hashes_full[full_hash] = filename
            except (OSError,):
                # the file access might've changed till the exec point got here
                continue


if __name__ == "__main__":
    if sys.argv[1:]:
        check_for_duplicates(["/content/Thai_ttf"])
    else:
        print("Please pass the paths to check as parameters to the script")

0it [00:00, ?it/s]
100%|██████████| 2334/2334 [00:00<00:00, 43814.64it/s]
1it [00:00, 14.87it/s]


idx 1 Duplicate found: /content/Thai_ttf/JS-Chawlewhieng.ttf and /content/Thai_ttf/JS Chawlewhieng Regular.ttf
idx 2 Duplicate found: /content/Thai_ttf/TH-Chakra-Petch-Bold.ttf and /content/Thai_ttf/TH Chakra Petch Bold.ttf
idx 3 Duplicate found: /content/Thai_ttf/ChulaNarak-Italic.ttf and /content/Thai_ttf/ChulaNarak Italic.ttf
idx 4 Duplicate found: /content/Thai_ttf/Athiti-SemiBold.ttf and /content/Thai_ttf/Athiti SemiBold.ttf
idx 5 Duplicate found: /content/Thai_ttf/Trirong Regular.ttf and /content/Thai_ttf/Trirong-Regular.ttf
idx 6 Duplicate found: /content/Thai_ttf/JS-Prasoplarp-Normal.ttf and /content/Thai_ttf/JS Prasoplarp Normal.ttf
idx 7 Duplicate found: /content/Thai_ttf/Trirong SemiBold Italic.ttf and /content/Thai_ttf/Trirong-SemiBoldItalic.ttf
idx 8 Duplicate found: /content/Thai_ttf/Trirong SemiBold.ttf and /content/Thai_ttf/Trirong-SemiBold.ttf
idx 9 Duplicate found: /content/Thai_ttf/Maitree-Regular.ttf and /content/Thai_ttf/Maitree Regular.ttf
idx 10 Duplicate found: 

In [ ]:
len(os.listdir("/content/Thai_ttf/"))

2071

In [ ]:
!zip -r Thai_TTF.zip Thai_ttf

  adding: Thai_ttf/ (stored 0%)
  adding: Thai_ttf/MN HOITHOT.ttf (deflated 54%)
  adding: Thai_ttf/FCTahoMai-SemiBoldItalic.ttf (deflated 52%)
  adding: Thai_ttf/NotoSansThai-Thin.ttf (deflated 49%)
  adding: Thai_ttf/Tlwg Typo Oblique.ttf (deflated 49%)
  adding: Thai_ttf/MN Mu Sawan Bold Italic.ttf (deflated 51%)
  adding: Thai_ttf/SP-Paichueak.ttf (deflated 44%)
  adding: Thai_ttf/MN YaHom Italic.ttf (deflated 51%)
  adding: Thai_ttf/MN Pla Krop Italic.ttf (deflated 49%)
  adding: Thai_ttf/MN BAMI Italic.ttf (deflated 54%)
  adding: Thai_ttf/BooK_Handwrite[2006].ttf (deflated 45%)
  adding: Thai_ttf/SP-Laisan1.ttf (deflated 50%)
  adding: Thai_ttf/FC SaveSpace Rounded Italic.ttf (deflated 57%)
  adding: Thai_ttf/MN NAKHARAT Bold.ttf (deflated 56%)
  adding: Thai_ttf/MN Bakery.ttf (deflated 52%)
  adding: Thai_ttf/MN Kha Mu Light.ttf (deflated 59%)
  adding: Thai_ttf/sd startrek version2.ttf (deflated 61%)
  adding: Thai_ttf/NPK-Sumalee14.ttf (deflated 47%)
  adding: Thai_ttf/MN Don

In [ ]:
!mv /content/Thai_TTF.zip /content/drive/MyDrive/ThaiVecFont/Thai_TTF.zip